In [21]:
from equations import *
from equations_matrix import *
from solvers import solve_X_prime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import numpy as np

# Set dimensions
N = 5  # number of countries
J = 3  # number of sectors

# Generate dummy arrays for the exogenous inputs
w_hat = np.ones(N)
td_prime = np.zeros(N)

# pif_hat and pim_hat are (N, N, J)
pif_hat = np.ones((N, N, J)) / N
pim_hat = np.ones((N, N, J)) / N


# Define dummy classes for ModelParams and ModelShocks
class ModelParams:
    pass

class ModelShocks:
    pass

# Create instances
mp = ModelParams()
shocks = ModelShocks()

# Fill ModelParams with attributes
mp.w0 = np.ones(N)
mp.L0 = np.ones(N)
mp.alpha = np.ones((N, J)) / J
mp.pif = np.ones((N, N, J))
mp.pim = np.ones((N, N, J)) 
mp.gamma = np.ones((N, J, J)) * 0.7 / J

# Fill ModelShocks with attributes
# Ensure tilde_tau_prime > 1 to avoid division by zero issues
shocks.tilde_tau_prime = np.random.rand(N, N, J) + 1.1

In [39]:
Xf_init = mp.alpha * (w_hat * mp.w0 * mp.L0 + td_prime)[:, None]  # shape: (N, J)
Xm_init = np.zeros_like(Xf_init)                                   # shape: (N, J)

# Call your iterative solver (solve_X_prime) using the guess:
Xf_iter, Xm_iter = solve_X_prime(w_hat, pif_hat, pim_hat, td_prime, Xf_init, Xm_init, mp, shocks, mute=False)

X_prime converged in 59 iterations


In [40]:
# Call the closed-form solver (calc_X) to get the direct solution:
Xf_closed, Xm_closed = calc_X(w_hat, pif_hat, pim_hat, td_prime, mp, shocks)

In [41]:
# Compare the results between the iterative and closed-form solutions.
print("Difference in Xf (iterative vs. closed-form):", np.linalg.norm(Xf_iter - Xf_closed))
print("Difference in Xm (iterative vs. closed-form):", np.linalg.norm(Xm_iter - Xm_closed))

Difference in Xf (iterative vs. closed-form): 1.3843184307588144e-05
Difference in Xm (iterative vs. closed-form): 1.6239500139258548e-05
